# 客户端

客户端代码与实现以下功能的服务器交互：

* 为代理定制流式传输
* 用户选定工具的代理

此代理没有记忆功能！要查看如何添加记忆功能，请参考 LangServe 中的其他示例。

**注意** 我们让代理以字符串形式流式传输输出。这几乎肯定不是您的应用程序所希望的。请随意调整以返回更结构化的输出；然而，请记住，客户端可能只需要使用 `astream_events`！

请参阅有关代理的相关文档：

* 创建自定义代理：https://python.langchain.com/docs/modules/agents/how_to/custom_agent 
* 使用代理进行流式传输：https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events 
* 通用流式文档：https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [13]:
word = "好啊啊啊"
len(word)

4

In [14]:
import requests

inputs = {"input": {"input": f"‘{word}’这个单词的长度是多少？", "chat_history": [], "tools": []}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

print(response.json()['output'])


启动代理: agent 输入: {'input': '‘好啊啊啊’这个单词的长度是多少？'}
<tool_call>
{"id": 0, "name": "calculateWordLength", "arguments": {"word": "\u597d\u3042\u3042\u3042"}}
</tool_call>
完成代理: agent 输出: <tool_call>
{"id": 0, "name": "calculateWordLength", "arguments": {"word": "\u597d\u3042\u3042\u3042"}}
</tool_call>



Let's provide it with a tool to test that tool selection works

In [15]:
import requests

inputs = {"input": {"input": f"‘{word}’这个单词的长度是多少？",  "chat_history": [], "tools": ["word_length", "favorite_animal"]}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

print(response.json()['output'])


启动代理: agent 输入: {'input': '‘好啊啊啊’这个单词的长度是多少？'}

启动工具: word_length 输入: {'word': '好一业业'}

完成工具: word_length 输出: 4
The length of the word "好啊啊啊" is 4 characters.
完成代理: agent 输出: The length of the word "好啊啊啊" is 4 characters.



您还可以通过 RemoteRunnable 接口与之交互（用于其他链路中）。

In [16]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

远程可运行对象与本地可运行对象具有相同的接口。

## 流

从**自定义流式传输**实现中流式传输输出，该实现流式传输中间步骤的字符串表示形式。详情请参见服务器端实现。

In [18]:
async for chunk in remote_runnable.astream({"input": "吴磊（Loi）最喜欢的动物是什么？", "tools": ["word_length"]}):
    print(chunk, end='|', flush=True)


|

启动代理: agent 输入: {'input': '吴磊（Loi）最喜欢的动物是什么？'}|
|Lo|i| Wu|（|吴|磊|）|最|喜欢|的|动物|是|猫|。|
|完成代理: agent 输出: Loi Wu（吴磊）最喜欢的动物是猫。|
|

In [19]:
async for chunk in remote_runnable.astream({"input": "吴磊（Loi）最喜欢的动物是什么？", "tools": ["word_length", "favorite_animal"]}):
    print(chunk, end='|', flush=True)


|启动代理: agent 输入: {'input': '吴磊（Loi）最喜欢的动物是什么？'}|
|


|启动工具: favorite_animal 输入: {'name': '瓦紎'}|
|
|完成工具: favorite_animal 输出: dog|
|吴|磊|（|Lo|i|）|最|喜欢|的|动物|是|狗|。|
|完成代理: agent 输出: 吴磊（Loi）最喜欢的动物是狗。|
|